In [21]:
import nipype.pipeline.engine as pe

import nipype.interfaces.fsl as fsl
import nipype.interfaces.io as nio

In [22]:
import glob

In [23]:
subject_ids = [fn.split('/')[-1][:] for fn in glob.glob('/home/gdholla1/data/daphne/clean/*')]

In [24]:
def extract_weighted_timeseries(data_fn, mask_fn):
    import nibabel as nb
    import numpy as np
    from nipype.utils.filemanip import split_filename
    import os
    
    data_image = nb.load(data_fn)
    mask_image = nb.load(mask_fn)
    
    data = data_image.get_data()
    mask = mask_image.get_data()
 
    
    ts = (data[mask > 0] * mask[mask > 0][:, np.newaxis]  / mask.sum()).sum(0)
    
    _, fn, _ = split_filename(data_fn)
    new_fn = os.path.abspath('%s_weighted.txt' % fn)
    
    np.savetxt(new_fn, ts)
    
    return new_fn
    

In [25]:
fn = extract_weighted_timeseries('/home/gdholla1/data/daphne/preprocess_phys/motion_regressors_filtered_files/_subject_id_S00/_fwhm_0.0/_addmean0/run1_maths_maths_maths_dtype_mcf_mask_gms_tempfilt_regfilt_maths.nii.gz',
                            '/home/gdholla1/data/daphne/registration/conjunct_masks/_mask_LC_L_subject_id_S00/_interp_trilinear/LC_L_conjunct_flirt.nii.gz')

In [26]:
import nipype.interfaces.utility as util

In [27]:
templates = {'slab2tse_mat':'/home/gdholla1/data/daphne/registration/slab2tse/_subject_id_{subject_id}/run1_dtype_mcf_mask_gms_mean_flirt_whole_brain_epi_flirt_tse_flirt_inv.mat',
             'conjunct_mask':'/home/gdholla1/data/daphne/masks/tse_space/{subject_id}/{mask}_conjunct.nii.gz',
             'disjunct_mask':'/home/gdholla1/data/daphne/masks/tse_space/{subject_id}/{mask}_disjunct.nii.gz',
              'tsnr':'/home/gdholla1/data/daphne/preprocessed/tsnr_file/_subject_id_{subject_id}/_fwhm_0.0/_tsnr*/run*_dtype_mcf_mask_gms_tempfilt_maths_tsnr.nii.gz',
             'mean_epi':'/home/gdholla1/data/daphne/preprocessed/feat_preprocess/mean/_subject_id_{subject_id}/_fwhm_0.0/run1_dtype_mcf_mask_gms_mean.nii.gz',
             'tsnr_phys':'/home/gdholla1/data/daphne/preprocess_phys/tsnr_file/_subject_id_{subject_id}/_fwhm_0.0/_tsnr*/run*_maths_maths_maths_dtype_mcf_mask_gms_tempfilt_maths_tsnr.nii.gz',
             'epi_filtered':'/home/gdholla1/data/daphne/preprocessed/motion_regressors_filtered_files/_subject_id_{subject_id}/_fwhm_0.0/_addmean*/run*_dtype_mcf_mask_gms_tempfilt_regfilt_maths.nii.gz',
             'epi_filtered_phys':'/home/gdholla1/data/daphne/preprocess_phys/motion_regressors_filtered_files/_subject_id_{subject_id}/_fwhm_0.0/_addmean*/run*_maths_maths_maths_dtype_mcf_mask_gms_tempfilt_regfilt_maths.nii.gz'}

workflow = pe.Workflow(name='transform_masks', base_dir='/home/gdholla1/workflow_folders/')

selector = pe.Node(nio.SelectFiles(templates), name='selector')

selector.iterables = [('subject_id', subject_ids), ('mask', ['LC_L', 'LC_R'])]

inverter = pe.Node(fsl.ConvertXFM(), name='inverter')
inverter.inputs.invert_xfm = True

workflow.connect(selector, 'slab2tse_mat', inverter, 'in_file')


# TRANSFORM DISJUNCT
transform_disjunct = pe.Node(fsl.ApplyXfm(), name='transform_disjunct')
transform_disjunct.iterables = [('interp', ['trilinear', 'nearestneighbour'])]

workflow.connect(selector, 'disjunct_mask', transform_disjunct, 'in_file')
workflow.connect(inverter, 'out_file', transform_disjunct, 'in_matrix_file')
workflow.connect(selector, 'mean_epi', transform_disjunct, 'reference')

# TRANSFORM CONJUNCT
transform_conjunct = transform_disjunct.clone('transform_conjunct')
transform_conjunct.iterables = [('interp', ['trilinear', 'nearestneighbour'])]

workflow.connect(selector, 'conjunct_mask', transform_conjunct, 'in_file')
workflow.connect(inverter, 'out_file', transform_conjunct, 'in_matrix_file')
workflow.connect(selector, 'mean_epi', transform_conjunct, 'reference')

# EXTRACT TSNR DISJUNCT
tsnr_disjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='tsnr_disjunct')
workflow.connect(selector, 'tsnr', tsnr_disjunct, 'in_file')
workflow.connect(transform_disjunct, 'out_file', tsnr_disjunct, 'mask')

# EXTRACT TSNR CONJUNCT
tsnr_conjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='tsnr_conjunct')
workflow.connect(selector, 'tsnr', tsnr_conjunct, 'in_file')
workflow.connect(transform_conjunct, 'out_file', tsnr_conjunct, 'mask')

# EXTRACT TSNR_PHYS DISJUNCT
tsnr_phys_disjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='tsnr_phys_disjunct')
workflow.connect(selector, 'tsnr_phys', tsnr_phys_disjunct, 'in_file')
workflow.connect(transform_disjunct, 'out_file', tsnr_phys_disjunct, 'mask')

# EXTRACT TSNR_PHYS CONJUNCT
tsnr_phys_conjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='tsnr_phys_conjunct')
workflow.connect(selector, 'tsnr_phys', tsnr_phys_conjunct, 'in_file')
workflow.connect(transform_disjunct, 'out_file', tsnr_phys_conjunct, 'mask')


# EXTRACT SIGNAL DISJUNCT
extract_signal_disjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='extract_signal_disjunct')
workflow.connect(selector, 'epi_filtered', extract_signal_disjunct, 'in_file')
workflow.connect(transform_disjunct, 'out_file', extract_signal_disjunct, 'mask')

# EXTRACT SIGNAL CONJUNCT
extract_signal_conjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='extract_signal_conjunct')
workflow.connect(selector, 'epi_filtered', extract_signal_conjunct, 'in_file')
workflow.connect(transform_conjunct, 'out_file', extract_signal_conjunct, 'mask')

# EXTRACT SIGNAL_PHYS DISJUNCT
extract_signal_phys_disjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='extract_signal_phys_disjunct')
workflow.connect(selector, 'epi_filtered_phys', extract_signal_phys_disjunct, 'in_file')
workflow.connect(transform_disjunct, 'out_file', extract_signal_phys_disjunct, 'mask')

# EXTRACT SIGNAL_PHYS CONJUNCT
extract_signal_phys_conjunct = pe.MapNode(fsl.ImageMeants(), iterfield=['in_file'], name='extract_signal_phys_conjunct')
workflow.connect(selector, 'epi_filtered_phys', extract_signal_phys_conjunct, 'in_file')
workflow.connect(transform_conjunct, 'out_file', extract_signal_phys_conjunct, 'mask')

# EXTRACT WEIGHTED SIGNAL PHYS CONJUNCT
extract_weighted_phys = pe.MapNode(util.Function(function=extract_weighted_timeseries,
                                                 input_names=['data_fn', 'mask_fn'],
                                                 output_names=['weighted_timeseries']),
                                   iterfield=['data_fn'],
                                   name='extract_weighted_phys')
workflow.connect(selector, 'epi_filtered_phys', extract_weighted_phys, 'data_fn')
workflow.connect(transform_conjunct, 'out_file', extract_weighted_phys, 'mask_fn')

ds = pe.Node(nio.DataSink(base_directory='/home/gdholla1/data/daphne/registration/'), name='datasink')

workflow.connect(transform_disjunct, 'out_file', ds, 'disjunct_masks')
workflow.connect(transform_conjunct, 'out_file', ds, 'conjunct_masks')
workflow.connect(tsnr_disjunct, 'out_file', ds, 'disjunct_tsnr_values.unfiltered')
workflow.connect(tsnr_conjunct, 'out_file', ds, 'conjunct_tsnr_values.unfiltered')

workflow.connect(tsnr_phys_disjunct, 'out_file', ds, 'disjunct_tsnr_values.phys_filtered')
workflow.connect(tsnr_phys_conjunct, 'out_file', ds, 'conjunct_tsnr_values.phys_filtered')

workflow.connect(extract_signal_disjunct, 'out_file', ds, 'extracted_signal_disjunct.unfiltered')
workflow.connect(extract_signal_conjunct, 'out_file', ds, 'extracted_signal_conjunct.unfiltered')

workflow.connect(extract_signal_phys_disjunct, 'out_file', ds, 'extracted_signal_disjunct.phys_filtered')
workflow.connect(extract_signal_phys_conjunct, 'out_file', ds, 'extracted_signal_conjunct.phys_filtered')

workflow.connect(extract_weighted_phys, 'weighted_timeseries', ds, 'extracted_signal_weighted')

workflow.run()

In [ ]:
/home/gdholla1/data/daphne/registration/extracted_signal_weighted/_mask_LC_L_subject_id_S00/_interp_trilinear/_extract_weighted_phys0/run1_maths_maths_maths_dtype_mcf_mask_gms_tempfilt_regfilt_maths_weighted.txt